# 🧠 Emotion Classifier – Fine-tuning DistilBERT

Ce projet fine-tune le modèle `distilbert-base-uncased` sur le dataset **Emotion** pour classifier les émotions dans un texte.

## 🎯 Objectifs
- Utiliser un modèle pré-entraîné (DistilBERT)
- Fine-tuner sur le dataset *Emotion*
- Sauvegarder le modèle et le tester sur un texte personnel
- Documenter les résultats


In [ ]:
!pip install transformers datasets torch

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from datasets import load_dataset
import torch

## 📥 Étape 3 : Chargement du dataset *Emotion*

On utilise le dataset `emotion` disponible sur 🤗 HuggingFace.


In [ ]:
print("Chargement du dataset Emotion...")
dataset = load_dataset("emotion")

print("\nExemple :")
print(dataset["train"][0])
print("\nLabels disponibles :", dataset["train"].features["label"].names)

Chargement du dataset Emotion...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]


Exemple :
{'text': 'i didnt feel humiliated', 'label': 0}

Labels disponibles : ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


## ✂️ Étape 4 : Tokenisation des textes

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

print("\nTokenisation en cours...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


Tokenisation en cours...


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## 🧹 Étape 5 : Préparation des jeux de données
On garde les colonnes utiles et on renomme les labels.

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

## 🧠 Étape 6 : Chargement du modèle pré-entraîné


In [ ]:
num_labels = dataset["train"].features["label"].num_classes
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## ⚙️ Étape 7 : Configuration de l'entraînement


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./models/emotion-distilbert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"  # empêche tout logging externe (wandb, tensorboard)
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

/tmp/ipython-input-186356536.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## 🏋️ Étape 8 : Fine-tuning du modèle

In [ ]:
print("\nDébut du fine-tuning...")
trainer.train()


Début du fine-tuning...


Epoch,Training Loss,Validation Loss
1,0.206800,0.202165
2,0.132800,0.168041


TrainOutput(global_step=2000, training_loss=0.317655956029892, metrics={'train_runtime': 1631.7693, 'train_samples_per_second': 19.611, 'train_steps_per_second': 1.226, 'total_flos': 4239259140096000.0, 'train_loss': 0.317655956029892, 'epoch': 2.0})

## 📈 Étape 9 : Évaluation sur le jeu de test

In [ ]:
print("\nÉvaluation sur le jeu de test...")
metrics = trainer.evaluate()
print(metrics)


Évaluation sur le jeu de test...


{'eval_loss': 0.16804073750972748, 'eval_runtime': 34.1292, 'eval_samples_per_second': 58.601, 'eval_steps_per_second': 3.663, 'epoch': 2.0}


## 💾 Étape 10 : Sauvegarde du modèle fine-tuné

In [ ]:
print("\nSauvegarde du modèle...")
trainer.save_model("./models/emotion-distilbert")
tokenizer.save_pretrained("./models/emotion-distilbert")


Sauvegarde du modèle...


('./models/emotion-distilbert/tokenizer_config.json',
 './models/emotion-distilbert/special_tokens_map.json',
 './models/emotion-distilbert/vocab.txt',
 './models/emotion-distilbert/added_tokens.json',
 './models/emotion-distilbert/tokenizer.json')

## 💬 Étape 11 : Test du modèle sur un texte personnel

In [ ]:
emotion_classifier = pipeline(
    "text-classification",
    model="./models/emotion-distilbert",
    tokenizer="./models/emotion-distilbert"
)

test_text = "I'm so happy and excited today!"
print("\nTexte de test :", test_text)
result = emotion_classifier(test_text)
print("Résultat :", result)


Device set to use cuda:0



Texte de test : I'm so happy and excited today!
Résultat : [{'label': 'LABEL_1', 'score': 0.9971639513969421}]
